# Testing Jupyter 

In [9]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv("top5_daily_ohlcv_since_2018-02-09.csv")

# print(df.head())

def calculate_daily_returns(df) -> pd.Series:
    """
        Calculate the daily returns from a DataFrame with a 'Close' column.

        Parameters:
            df (pd.DataFrame): DataFrame containing historical price data with a 'Close' column.

        Returns:
            pd.Series: A Series of daily return values (as decimal percentages).
    """

    daily_returns = df['close'].pct_change()
    # .pct_change() is a built in pandas method 
    # ( calculates the percentage change between the current and the prev element)
    return daily_returns

df['timestamp'] = pd.to_datetime(df['timestamp'])

df['Daily Return'] = calculate_daily_returns(df)

In [10]:
def extract_date_components(timestamp_input) -> dict:
    """
    Transform a timestamp (string in “YYYY-MM-DD HH:MM:SS” format *or* a
    datetime.datetime object) into a dictionary of calendar-based features.

    Parameters
    ----------
    timestamp_input : str | datetime.datetime
        Either the timestamp string (e.g. "2023-10-26 14:30:00")
        or an already-parsed datetime object.

    Returns
    -------
    dict
        {
            "year":               2023,
            "quarter":            4,
            "month_number":       10,
            "month_name":         "October",
            "week_of_year":       43,      # ISO-8601 week number
            "day_of_week_number": 4,       # Monday=1 … Sunday=7  (ISO)
            "day_of_week_name":   "Thursday"
        }
    """

# Checks if timestamp input is of instance/type datetime
    if isinstance(timestamp_input, datetime):
        dt = timestamp_input
    elif isinstance(timestamp_input, str):
        try:   
            dt = datetime.strptime(timestamp_input, "%Y-%m-%d %H:%M:%S")
        except ValueError as exc:          
            raise ValueError(
                "Timestamp string must match 'YYYY-MM-DD HH:MM:SS'"
            ) from exc
    else:
        raise TypeError(
            "timestamp_input must be a datetime object or a timestamp string"
        )


    features = {
         "year":               dt.year,
        "quarter":            (dt.month - 1) // 3 + 1,          # 1–4
        "month_number":       dt.month,                         # 1–12
        "month_name":         dt.strftime("%B"),                # "January"
        "week_number":         dt.isocalendar().week,         # "January"
        "day_of_week_number": dt.weekday() + 1,                    # 1=Mon … 7=Sun
        "day_of_week_name":   dt.strftime("%A"),                # "Monday"…
    }

    return features

In [11]:
test_output = extract_date_components("2023-10-26 14:30:00")
print(test_output)


{'year': 2023, 'quarter': 4, 'month_number': 10, 'month_name': 'October', 'week_number': 43, 'day_of_week_number': 4, 'day_of_week_name': 'Thursday'}


In [12]:
from datetime import datetime

test_dt = datetime(2025, 5, 26, 20, 15, 0)
print(extract_date_components(test_dt))


{'year': 2025, 'quarter': 2, 'month_number': 5, 'month_name': 'May', 'week_number': 22, 'day_of_week_number': 1, 'day_of_week_name': 'Monday'}


In [13]:
# Expecting a ValueError due to bad format
try:
    extract_date_components("2023/10/26")
except ValueError as e:
    print("Caught expected ValueError:", e)

# Expecting a TypeError due to wrong input type
try:
    extract_date_components(12345)
except TypeError as e:
    print("Caught expected TypeError:", e)


Caught expected ValueError: Timestamp string must match 'YYYY-MM-DD HH:MM:SS'
Caught expected TypeError: timestamp_input must be a datetime object or a timestamp string


In [14]:
# Apply to a DataFrame and explode the dict into columns
df_features = df['timestamp'].apply(extract_date_components)

# Merge with the original DataFrame
df = pd.concat([df, df_features], axis=1)

# Preview the results
df.head()


,timestamp,open,high,low,close,volume,coin_id,Daily Return,timestamp
0,2018-02-09 00:00:00+00:00,7611.61,7611.61,7611.61,7611.61,3.190754e+09,bitcoin,NaN,"{'year': 2018, 'quarter': 1, 'month_number': 2..."
1,2018-02-10 00:00:00+00:00,8208.57,8674.76,7847.14,8672.57,2.960040e+09,bitcoin,0.139387,"{'year': 2018, 'quarter': 1, 'month_number': 2..."
2,2018-02-11 00:00:00+00:00,8659.92,9088.97,8283.43,8590.21,3.371678e+09,bitcoin,-0.009497,"{'year': 2018, 'quarter': 1, 'month_number': 2..."
3,2018-02-12 00:00:00+00:00,8583.38,8583.38,7890.82,8064.69,2.806997e+09,bitcoin,-0.061177,"{'year': 2018, 'quarter': 1, 'month_number': 2..."
4,2018-02-13 00:00:00+00:00,8105.98,8920.31,8105.98,8845.22,3.069220e+09,bitcoin,0.096784,"{'year': 2018, 'quarter': 1, 'month_number': 2..."


## Volatility metrics

In [15]:
def calculate_daily_returns(df) -> pd.Series:
    """
        Calculate the daily returns from a DataFrame with a 'Close' column.

        Parameters:
            df (pd.DataFrame): DataFrame containing historical price data with a 'Close' column.

        Returns:
            pd.Series: A Series of daily return values (as decimal percentages).
    """

    daily_returns = df['close'].pct_change()
    # .pct_change() is a built in pandas method 
    # ( calculates the percentage change between the current and the prev element)
    return daily_returns

df['timestamp'] = pd.to_datetime(df['timestamp'])

df['Daily Return'] = calculate_daily_returns(df)

ValueError: cannot assemble with duplicate keys

## 📊 Daily Returns (`calculate_daily_returns`)

### **Significance**
Daily returns measure the percentage change in asset price from one day to the next, providing the foundation for all volatility analysis.

### **Use Cases**
- **Risk assessment** - Understanding daily price movement magnitude
- **Performance measurement** - Calculating investment returns over time
- **Volatility calculation** - Input for standard deviation and other volatility metrics
- **Statistical analysis** - Foundation for probability distributions and risk models

### **Example**
```python
# Bitcoin price: Day 1 = $50,000, Day 2 = $52,000
# Daily return = (52,000 - 50,000) / 50,000 = 0.04 (4%)
```

In [34]:
def get_return_std_dev(return_series):
    """
    Calculates the standard deviation of returns from a Series of daily returns.    

    Parameters:
        return_series (pd.Series): Series containing daily returns.

    Returns:
        float: The standard deviation of returns, rounded to 4 decimal places.

    Example usage:
        std_dev = get_return_std_dev(daily_returns)
    """
    return round(return_series.std(), 4)

## 📈 Standard Deviation (`get_return_std_dev`)

### **Significance**
Standard deviation quantifies the dispersion of daily returns around their mean, providing a single number that captures overall volatility.

### **Use Cases**
- **Volatility measurement** - Single metric for asset risk assessment
- **Portfolio construction** - Comparing risk across different assets
- **Risk management** - Setting position sizes and stop losses
- **Performance evaluation** - Risk-adjusted return calculations

### **Example**
```python
# Bitcoin daily returns: [0.02, -0.01, 0.03, -0.02, 0.01]
# Standard deviation = 0.019 (1.9% daily volatility)
# Interpretation: 68% of days see moves within ±1.9%
```


In [37]:
def get_rolling_volatility(return_series, window):
    """
    Calculates the rolling standard deviation of returns over a specified window.

    Parameters:
        return_series (pd.Series): Series containing daily returns.
        window (int): The number of days to calculate the rolling standard deviation over.
    
    Returns:
        pd.Series: A Series containing the rolling standard deviation of returns.

    Example usage:
        rolling_volatility = get_rolling_volatility(daily_returns, 20)
    """
    return return_series.rolling(window).std()  


## 🔄 Rolling Volatility (`get_rolling_volatility`)

### **Significance**
Rolling volatility measures how volatility changes over time, capturing recent market behavior rather than long-term averages.

### **Use Cases**
- **Volatility regime detection** - Identifying periods of high/low volatility
- **Dynamic risk management** - Adjusting strategies based on current volatility
- **Market timing** - Entering/exiting positions based on volatility cycles
- **Trend confirmation** - Validating price movements with volatility context

### **Example**
```python
# 10-day rolling volatility: [2.1%, 2.3%, 1.8%, 3.2%, 2.9%]
# Interpretation: Volatility increased from 1.8% to 3.2% over 10 days
# Trading implication: Market stress increasing, consider reducing position sizes
```

In [21]:
return_series = calculate_daily_returns(df)

In [ ]:
print("STD: ", get_return_std_dev(return_series))

rolling volatility:  0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
          ...   
2647    0.022218
2648    0.022318
2649    0.023076
2650    0.022436
2651    0.021309
Name: close, Length: 2652, dtype: float64


In [ ]:
print("rolling volatility: ", get_rolling_volatility(return_series, window = 10))

<font color="blue"> HI </font>

In [21]:
def calculate_atr(df: pd.DataFrame, window) -> pd.Series:
    """
    Calculate the Average True Range (ATR) indicator.
    
    ATR measures volatility by calculating the average of the True Range over a specified period.
    True Range is the greatest of:
    - Current High - Current Low
    - |Current High - Previous Close|
    - |Current Low - Previous Close|
    
    Parameters:
        df (pd.DataFrame): DataFrame containing 'high', 'low', and 'close' columns
        window (int): Period for calculating the moving average (default: 14)
    
    Returns:
        pd.Series: The Average True Range values
    
    Example:
        atr = calculate_atr(df, window=14)
    """
    # Extract the required price columns
    high = df['high']
    low = df['low']
    close = df['close']
    
    # Calculate the three components of True Range
    # 1. Current High - Current Low
    high_low = high - low
    
    # 2. Absolute value of (Current High - Previous Close)
    high_prev_close = np.abs(high - close.shift(1))
    
    # 3. Absolute value of (Current Low - Previous Close)
    low_prev_close = np.abs(low - close.shift(1))
    
    # True Range is the maximum of the three values
    true_range = pd.concat([high_low, high_prev_close, low_prev_close], axis=1).max(axis=1)
    
    # Calculate ATR using Simple Moving Average
    # Note: Traditional ATR uses Wilder's smoothing (EMA), but SMA is also common
    atr_sma = true_range.rolling(window=window).mean()
    
    return atr_sma

## 📏 Average True Range (`calculate_atr`)

### **Significance**
ATR measures actual price volatility by considering gaps and intraday movement, providing a more comprehensive volatility measure than simple price changes.

### **Use Cases**
- **Stop loss placement** - Dynamic stops based on current volatility
- **Position sizing** - Adjusting position size based on market volatility
- **Breakout confirmation** - Validating price breakouts with volume/volatility
- **Trend strength assessment** - Measuring momentum and trend reliability

### **Example**
```python
# Bitcoin ATR: $2,500
# Interpretation: Average daily price range is $2,500
# Trading implication: Set stops at least $2,500 from entry price
```


In [15]:
high = df['high']
low = df['low']
close = df['close']
high_low = high - low
    
    # 2. Absolute value of (Current High - Previous Close)
high_prev_close = np.abs(high - close.shift(1))
    
    # 3. Absolute value of (Current Low - Previous Close)
low_prev_close = np.abs(low - close.shift(1))
true_range = pd.concat([high_low, high_prev_close, low_prev_close], axis=1).max(axis=1)

In [22]:
print("ATR: ", calculate_atr(df, window = 10))

ATR:  0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
         ...  
2647    2363.8
2648    2184.3
2649    2482.9
2650    2695.7
2651    2618.6
Length: 2652, dtype: float64


In [17]:
def calculate_volatility_ratio(df: pd.DataFrame, recent_window: int, long_window: int) -> pd.Series:
    """
    Calculate the volatility ratio of a DataFrame.
    
    The volatility ratio is calculated as the ratio of the standard deviation of the recent returns to the standard deviation of the long-term returns.
    
    Parameters:
        df (pd.DataFrame): DataFrame containing 'close' column
        recent_window (int): Window for recent returns
        long_window (int): Window for long-term returns
    
    Returns:            
        pd.Series: The volatility ratio values
    
    Example:
        volatility_ratio = calculate_volatility_ratio(df, recent_window=20, long_window=50)
    """
    # Calculate recent returns  
    recent_atr = get_rolling_volatility(df, recent_window)
    long_atr = get_rolling_volatility(df, long_window)

    # Calculate volatility ratio
    volatility_ratio = recent_atr / long_atr

    return volatility_ratio

## ⚖️ Volatility Ratio (`calculate_volatility_ratio`)

### **Significance**
Volatility ratio compares recent volatility to long-term volatility, identifying when market conditions are changing relative to historical norms.

### **Use Cases**
- **Regime shift detection** - Identifying when market behavior fundamentally changes
- **Risk adjustment** - Scaling positions based on volatility regime
- **Market timing** - Entering positions when volatility is low and expected to rise
- **Portfolio allocation** - Adjusting exposure based on volatility conditions

### **Example**
```python
# Volatility ratio: 1.5
# Interpretation: Recent volatility is 50% higher than long-term average
# Trading implication: Market stress increasing, reduce position sizes
```


In [18]:
print("Volatility Ratio: ", calculate_volatility_ratio(df, recent_window=20, long_window=50))


Volatility Ratio:  0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
          ...   
2647    0.874396
2648    0.837973
2649    0.789703
2650    0.805635
2651    0.794770
Length: 2652, dtype: float64


Understanding the Return Value
The ratio tells you:
Ratio > 1.0 = Recent volatility is higher than long-term volatility
Ratio = 1.0 = Recent volatility equals long-term volatility
Ratio < 1.0 = Recent volatility is lower than long-term volatility
Practical Interpretation
High Volatility Ratio (> 1.0):
Market stress - Recent price movements are more volatile than usual
Increased uncertainty - Market participants are more anxious
Potential opportunities - Higher volatility often creates trading opportunities
Risk warning - May indicate market instability or trend changes
Low Volatility Ratio (< 1.0):
Market calm - Recent price movements are less volatile than usual
Complacency - Market participants may be too comfortable
Potential complacency trap - Low volatility often precedes large moves
Range-bound markets - May indicate sideways consolidation

In [40]:
def calculate_bollinger_bands(df: pd.DataFrame, window: int, num_std_dev: int) -> pd.DataFrame:
    """
    Calculate Bollinger Bands for a DataFrame.
    
    Bollinger Bands are a technical analysis tool that consists of three lines:
    - Middle Band: Moving average of the closing prices
    - Upper Band: Middle Band + (num_std_dev * standard deviation of the closing prices)
    - Lower Band: Middle Band - (num_std_dev * standard deviation of the closing prices)

    Parameters:
        df (pd.DataFrame): DataFrame containing 'close' column
        window (int): Window for the moving average
        num_std_dev (int): Number of standard deviations to include in the bands

    Returns:
        pd.DataFrame: DataFrame containing the Bollinger Bands

    Example:
        bollinger_bands = calculate_bollinger_bands(df, window=20, num_std_dev=2)  

    """

    closing_prices = df['close']

    middle_band = closing_prices.rolling(window=window).mean()

    rolling_std = closing_prices.rolling(window=window).std()

    upper_band = middle_band + (num_std_dev * rolling_std)

    lower_band = middle_band - (num_std_dev * rolling_std)

    bollinger_bands = pd.DataFrame({
        'middle_band': middle_band,
        'upper_band': upper_band,
        'lower_band': lower_band
    })

    return bollinger_bands

## 📊 Bollinger Bands (`calculate_bollinger_bands`)

### **Significance**
Bollinger Bands create dynamic support/resistance levels based on price volatility, helping identify overbought/oversold conditions and potential breakouts.

### **Use Cases**
- **Overbought/oversold identification** - Price at band extremes suggests reversal
- **Breakout detection** - Price breaking through bands indicates trend continuation
- **Volatility measurement** - Band width indicates current volatility level
- **Mean reversion trading** - Trading price returns to the middle band

### **Example**
```python
# Bitcoin price: $50,000
# Upper band: $52,000, Lower band: $48,000, Middle band: $50,000
# Interpretation: Price at middle band, normal volatility
# Trading implication: No extreme conditions, follow trend direction
```

In [41]:
def calculate_bollinger_bands_width(df: pd.DataFrame, window: int, num_std_dev: int) -> pd.Series:
    """
    Calculate the width of Bollinger Bands.
    
    The width is calculated as the difference between the upper and lower bands divided by the middle band.

    Parameters:
        df (pd.DataFrame): DataFrame containing 'close' column
        window (int): Window for the moving average
        num_std_dev (int): Number of standard deviations to include in the bands

    Returns:
        pd.Series: The width of the Bollinger Bands                                                                 

    Example:
        bollinger_bands_width = calculate_bollinger_bands_width(df, window=20, num_std_dev=2)
    """
    bollinger_bands = calculate_bollinger_bands(df, window, num_std_dev)

    return (bollinger_bands['upper_band'] - bollinger_bands['lower_band']) / bollinger_bands['middle_band']

## 📏 Bollinger Bands Width (`calculate_bollinger_bands_width`)

### **Significance**
Bollinger Bands width measures relative volatility by comparing current price ranges to recent history, providing context for volatility interpretation.

### **Use Cases**
- **Volatility regime identification** - Detecting when volatility is expanding/contracting
- **Breakout confirmation** - Width expansion validates price breakouts
- **Squeeze pattern detection** - Narrow width often precedes major moves
- **Risk management** - Adjusting strategies based on volatility context

### **Example**
```python
# BB Width: 0.08 (8%)
# Interpretation: Current volatility is 8% of price level
# Trading implication: Moderate volatility, normal trading conditions
```


In [46]:
print("Bollinger Bands Width: ", calculate_bollinger_bands_width(df, window=3, num_std_dev=1))


Bollinger Bands Width:  0            NaN
1            NaN
2       0.142365
3       0.078120
4       0.093647
          ...   
2647    0.018002
2648    0.016208
2649    0.017102
2650    0.013621
2651    0.012642
Length: 2652, dtype: float64


In [20]:
print("Bollinger Bands: ", calculate_bollinger_bands(df, window=20, std_dev=2))


Bollinger Bands:        middle_band    upper_band    lower_band
0             NaN           NaN           NaN
1             NaN           NaN           NaN
2             NaN           NaN           NaN
3             NaN           NaN           NaN
4             NaN           NaN           NaN
...           ...           ...           ...
2647     95904.90  1.506176e+07 -1.486995e+07
2648     96732.00  1.402575e+07 -1.383229e+07
2649     97197.05  1.526176e+07 -1.506737e+07
2650     97726.00  1.685802e+07 -1.666257e+07
2651     98212.05  1.764076e+07 -1.744434e+07

[2652 rows x 3 columns]


## 🎯 Key Relationships Between Metrics

### **Complementary Analysis**
- **Daily Returns + Standard Deviation** = Overall risk profile
- **Rolling Volatility + Volatility Ratio** = Volatility regime changes
- **ATR + Bollinger Bands** = Dynamic support/resistance with volatility context
- **BB Width + Volatility Ratio** = Comprehensive volatility regime analysis

### **Trading Strategy Integration**
1. **Use standard deviation** for initial risk assessment
2. **Monitor rolling volatility** for regime changes
3. **Apply ATR** for dynamic position management
4. **Use volatility ratio** for strategic allocation decisions
5. **Combine with BB width** for entry/exit timing

### **Risk Management Hierarchy**
1. **Position sizing** based on standard deviation
2. **Stop placement** using ATR
3. **Entry timing** with Bollinger Bands
4. **Regime adjustment** via volatility ratio
5. **Volatility confirmation** through BB width 

## Volume Metrics

In [3]:
def volume_change(df: pd.DataFrame) -> pd.Series:
    """
    Calculate the volume change of a DataFrame.
    
    The volume change is calculated as the ratio of the current volume to the previous volume.
    
    Parameters:
        df (pd.DataFrame): DataFrame containing 'volume' column
    
    Returns:
        pd.Series: The volume change values
    """

    volume_change = df["volume"].pct_change()

    return volume_change

## 📊 Volume Change (`volume_change`)

### **Significance**
Volume change measures the percentage change in trading volume from one period to the next, providing insight into market participation dynamics and momentum shifts.

### **Use Cases**
- **Momentum confirmation** - Increasing volume validates price movements
- **Divergence detection** - Price up with volume down suggests weak moves
- **Breakout validation** - High volume changes confirm genuine breakouts
- **Market exhaustion** - Declining volume changes may signal trend end
- **Liquidity assessment** - Understanding market depth and participation

### **Example**
```python
# Volume change: +0.25 (25%)
# Interpretation: Volume increased 25% from previous period
# Trading implication: Strong market participation supporting current price action

# Volume change: -0.15 (-15%)
# Interpretation: Volume decreased 15% from previous period
# Trading implication: Declining participation, potential trend weakness
```

In [6]:
print("Volume Change: ", volume_change(df))

Volume Change:  0             NaN
1       -0.072307
2        0.139065
3       -0.167478
4        0.093418
           ...   
12628   -0.314860
12629    0.737151
12630   -0.659684
12631    0.660965
12632    0.305590
Name: volume, Length: 12633, dtype: float64


In [8]:
def relative_volume_change(df: pd.DataFrame, window: int) -> pd.Series:
    """
    Calculate the relative volume change of a DataFrame.
    
    The relative volume change is calculated as the ratio of the current volume to the previous volume.
    
    Parameters:
        df (pd.DataFrame): DataFrame containing 'volume' column
        window (int): Window for the moving average
    
    Returns:
        pd.Series: The relative volume change values
    """

    N_day_avg_volume = df["volume"].rolling(window=window).mean()
    current_volume = df["volume"]

    relative_volume_change = current_volume / N_day_avg_volume

    return relative_volume_change
    

## 📈 Relative Volume Change (`relative_volume_change`)

### **Significance**
Relative volume change compares current volume to its recent average, providing context for whether current trading activity is above or below normal levels.

### **Use Cases**
- **Volume spike detection** - Identifying unusual trading activity
- **Breakout confirmation** - High relative volume validates price breakouts
- **Market interest assessment** - Understanding if current activity is normal or exceptional
- **Trend strength validation** - Strong trends should show above-average volume
- **Reversal signals** - Volume spikes often precede major reversals

### **Interpretation**
- **Ratio > 1.0** = Current volume above average → High market interest
- **Ratio = 1.0** = Current volume equals average → Normal market activity
- **Ratio < 1.0** = Current volume below average → Low market interest

### **Example**
```python
# Relative volume: 1.5
# Interpretation: Current volume is 50% above the 5-day average
# Trading implication: Exceptional market interest, strong move likely

# Relative volume: 0.7
# Interpretation: Current volume is 30% below the 5-day average
# Trading implication: Low participation, weak move or consolidation
```

In [10]:
print("relative_volume_change", relative_volume_change(df, 5))

relative_volume_change 0             NaN
1             NaN
2             NaN
3             NaN
4        0.996585
           ...   
12628    0.934191
12629    1.325564
12630    0.481865
12631    0.850919
12632    1.146226
Name: volume, Length: 12633, dtype: float64


In [17]:
def calculate_volume_trend(df: pd.DataFrame, window: int) -> pd.Series:
    """
    Calculate the volume trend of a DataFrame.
    
    The volume trend is calculated as slope of linear regression of volume over time.
    
    Parameters:
        df (pd.DataFrame): DataFrame containing 'volume' column
        window (int): Window for the moving average
    
    Returns:    
        pd.Series: The volume trend values
    """
    
    def calculate_slope(values):
        """Calculate the slope of linear regression for a series of values."""
        if len(values) < 2 or values.isna().any():
            return np.nan
        
        # Create x values (time indices)
        x = np.arange(len(values))
        
        # Fit linear regression (degree 1 polynomial)
        # polyfit returns [slope, intercept]
        coefficients = np.polyfit(x, values, 1)
        
        # Return the slope (first coefficient)
        return coefficients[0]
    
    # Apply rolling window and calculate slope for each window
    volume_trend = df["volume"].rolling(window=window).apply(calculate_slope, raw=False)
    
    return volume_trend
    

## 📈 Volume Trend (`calculate_volume_trend`)

### **Significance**
Volume trend measures the direction and strength of volume changes over time using linear regression slope. This indicates whether market interest is strengthening (positive slope) or weakening (negative slope).

### **Use Cases**
- **Market interest assessment** - Determining if participation is increasing or decreasing
- **Trend confirmation** - Strong price trends should be accompanied by increasing volume
- **Divergence detection** - Price up with volume down = potential weakness
- **Breakout validation** - True breakouts show increasing volume trends
- **Market exhaustion** - Declining volume trends may signal end of moves

### **Interpretation**
- **Positive slope** = Volume increasing over time → Growing market interest
- **Negative slope** = Volume decreasing over time → Waning market interest
- **Slope magnitude** = Rate of change in market participation

### **Example**
```python
# Volume trend slope: +50,000 per day
# Interpretation: Volume is increasing by 50,000 units per day
# Trading implication: Strong market interest supporting the current trend

# Volume trend slope: -30,000 per day
# Interpretation: Volume is decreasing by 30,000 units per day  
# Trading implication: Market interest fading, potential trend exhaustion
```

In [18]:
# Test the volume trend calculation
volume_trend_20 = calculate_volume_trend(df, window=20)
volume_trend_50 = calculate_volume_trend(df, window=50)

print("Volume Trend (20-day window):")
print(volume_trend_20.tail(10))
print("\nVolume Trend (50-day window):")
print(volume_trend_50.tail(10))


Volume Trend (20-day window):
12623   -1.475384e+08
12624   -1.655589e+08
12625   -1.556525e+08
12626   -1.019919e+08
12627   -4.682997e+07
12628    2.666698e+07
12629    6.866727e+07
12630    4.647051e+07
12631    3.754462e+07
12632    5.957173e+07
Name: volume, dtype: float64

Volume Trend (50-day window):
12623   -5.867641e+05
12624   -9.313247e+06
12625   -1.292543e+07
12626   -9.334225e+06
12627   -3.052474e+06
12628   -1.216015e+06
12629    5.423030e+06
12630    2.475322e+06
12631   -3.035335e+06
12632   -6.192634e+06
Name: volume, dtype: float64
